<a href="https://www.kaggle.com/code/najeebz/bank-churn-gridsearchcv-xgboost-rf-abc-gbc?scriptVersionId=168911141" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# [Binary Classification with a Bank Churn Dataset](https://www.kaggle.com/competitions/playground-series-s4e1/overview)
### Playground Series - Season 4, Episode 1
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- Prepared for the submission to the competition.
________________________________________________________________________
# Attributions:


[Walter Reade, Ashley Chow. (2024). Binary Classification with a Bank Churn Dataset . Kaggle.](https://kaggle.com/competitions/playground-series-s4e1)
________________________________________________________________________
​
This Notebook is to be submitted to the competition so aims to start the process from the beginning to the creation of the submission csv file in proper format.
__________________________________________________________________________
# Code Execution and Versioning Repository: 
- [Execute the notebook in Kaggle](https://www.kaggle.com/code/najeebz/bank-churn-gridsearchcv-xgboost-gradient-boost)
- [Github Repository](https://github.com/snajeebz/playground)
​
____________________________________________________________________
# Citation:

Najeeb Zaidi. (2024). Binary Classification with a Bank Churn Dataset. Competition Submission. Kaggle. https://www.kaggle.com/code/najeebz/bank-churn-gridsearchcv-xgboost-gradient-boost

# Initiating the Notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from warnings import filterwarnings;
filterwarnings('ignore');

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the Dataset

In [ ]:
df=pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

In [ ]:
test

In [ ]:
df

# Analyzing the data

In [ ]:
test.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.Geography.value_counts()

In [ ]:
df.NumOfProducts.value_counts()


In [ ]:
df.IsActiveMember.describe()

In [ ]:
df

# Visualization

In [ ]:
from wordcloud import WordCloud, STOPWORDS

text = df['Surname'].values 
plt.figure(figsize=[15,7])
wc = WordCloud().generate(str(text))
plt.imshow(wc)

In [ ]:
train_df=df[['CreditScore','Geography','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Exited']]
train_df
test_df=test[['CreditScore','Geography','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']]

In [ ]:
cat_cols = df[['Geography','Gender','Tenure','NumOfProducts','HasCrCard','IsActiveMember','Exited']]
num_cols= train_df[['CreditScore', 'Age','Balance','EstimatedSalary']]

## Categorical Count Plots:

In [ ]:
for col in cat_cols:
    plt.figure(figsize=[15,7])
    sns.countplot(df,x=df[col])
    plt.show()

## Histograms of numerical columns

In [ ]:
for col in num_cols:
    plt.figure(figsize=[10,7])
    sns.distplot(df[col],kde=True)
    plt.axvline(df[col].mean(),color='r', label='Mean')
    plt.axvline(df[col].median(),color='y', linestyle='--',label='Median')
    plt.legend()
    plt.show()

## Analyzing the Data Hierarchy

In [ ]:
import plotly.express as px
fig = px.sunburst(
    df,
    path=['Geography','Gender','NumOfProducts','IsActiveMember','Exited'], 
    color='Exited',color_discrete_map={'1':'gold', '0':'darkblue'},
    width=1200, height=1200
)
fig.show()

## Relationships between different columns

In [ ]:
plt.figure(figsize=(15, 5))
s=sns.barplot(train_df,x='NumOfProducts',y='Exited',hue='Gender')
plt.figure(figsize=(15, 5))
s=sns.barplot(train_df,x='NumOfProducts',y='Exited',hue='Geography', palette='Greens')
plt.figure(figsize=(15, 5))
s=sns.barplot(train_df,y='Age',x='Exited',hue='Geography',palette='Blues')
plt.figure(figsize=(15, 5))
s=sns.barplot(train_df,x='Tenure',y='Exited',hue='Geography',palette='Reds')
plt.figure(figsize=(15, 5))
s=sns.scatterplot(train_df,y='Balance',x='Geography',hue='Exited',palette='crest')

# Pre Processing and Features Generation

In [ ]:
def colchange(train_df):
    train_df['Geography']=pd.to_numeric(train_df['Geography'].replace(['France','Spain', 'Germany'],[1,2,3]))
    train_df['Gender']=pd.to_numeric(train_df['Gender'].replace(['Male','Female'],[1,0]))
    return(train_df)
train_df=colchange(train_df)
test_df=colchange(test_df)
train_df['Exited']=pd.Categorical(train_df['Exited'])
print(test_df.info())
train_df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = train_df.corr()
# plot the heatmap
plt.figure(figsize=(20, 20))
s=sns.heatmap(corr,annot=True, cmap='crest')
corr = test_df.corr()
# plot the heatmap
plt.figure(figsize=(20, 20))
s=sns.heatmap(corr,annot=True, cmap='Blues')

In [ ]:
train_df

# Test and Train Split

In [ ]:
X=train_df.drop(columns=['Exited'])
y=(train_df[['Exited']].astype('int64'))

print(X.describe())
print(y.info())

## Scaling the Input

In [ ]:
def scale(X):
    from sklearn import preprocessing
    scaled=preprocessing.StandardScaler()
    scaler=scaled.fit(X)
    X=scaler.transform(X)
    return X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(scale(X),y,train_size=0.8, random_state=42)
X.keys()
test_df=scale(test_df)

In [ ]:
y.info()

# Training The Models

In [ ]:
def evaluate(y_test,ypred):
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 10))
    s=sns.heatmap(cm,annot=True, cmap='Reds')


# HyperParameter Tuning

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
result=pd.DataFrame(columns=['id', 'RF','MLP','XGB','ABC','GBC','Exited'])
result.id=test[['id']]
w=pd.DataFrame(columns=['mod','weight'])
models = {
   'RF': RandomForestClassifier(),
   'XGB' : xgb.XGBClassifier(objective="binary:logistic",seed=42),
   'ABC': AdaBoostClassifier(),
   'GBC': GradientBoostingClassifier(validation_fraction=0.1, n_iter_no_change=15),
   'MLP':MLPClassifier() 
}
grid_param={
   'MLP': {
       'solver': ['lbfgs'], 
       'early_stopping':[1],
       'activation': ['tanh'], 
       'max_iter': [500], 
       'alpha': [10.0 ** -7], 
       'hidden_layer_sizes':[12], 
       'learning_rate': ['adaptive'],
       'verbose':[0] 
          },
   'RF' : {
       'bootstrap': [True], 
       'ccp_alpha': [0.0001],
       'max_depth': [None], 
       'max_leaf_nodes': [None], 
       'max_samples': [None], 
       'min_impurity_decrease': [0.0], 
       'min_samples_leaf': [5], 
       'min_samples_split': [3], 
       'n_estimators': [70], 
       'warm_start': [False] 
        },
#{'bootstrap': True, 'ccp_alpha': 0.0001, 'max_depth': None, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 70, 'warm_start': False}
    'XGB': {
        'n_estimators':[5000],
        'learning_rate': [0.001],
        'max_depth': [5],
        'subsample': [0.1],
        'colsample_bytree':[0.1],
        'min_child_weight':[1],
        'alpha': 10.0 ** -np.arange(8,10,1)
          },
   'GBC': {
        'loss':['exponential'],
        'criterion': ['friedman_mse'],
        'n_estimators': [1000],
        'learning_rate': [0.2],
        'min_impurity_decrease':[0.1],
        'ccp_alpha': [0.03],
        'subsample': [1]
        },
  'ABC':  {
        'n_estimators':[1000],
        'learning_rate':[0.1]
        },
    #Model:  GBC  Accuracy: 0.79  Best Parameters: {'ccp_alpha': 0.03, 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'exponential', 'min_impurity_decrease': 0.1, 'n_estimators': 1000, 'subsample': 1}
}
for name,mod in tqdm(models.items()):
   gs = GridSearchCV(estimator=mod, param_grid=grid_param[name], verbose=3,cv=3, scoring='accuracy')
   gs.fit(X_train,np.ravel(y_train))
   print('Model: ', name, ' Accuracy:',format(gs.best_score_,".2f"),  ' Best Parameters:',gs.best_params_)
   y_pred=gs.predict(X_test)
   w.loc[len(w)]=[name,gs.best_score_]
   result[name]=gs.predict_proba(scale(test_df))[:,1]
   
   evaluate(y_test,y_pred)
# Model:  DT  Accuracy: 0.80  Best Parameters: {'ccp_alpha': 0.0, 'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 4, 'random_state': 42}
#Model:  RF  Accuracy: 0.86  Best Parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'max_depth': None, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 110, 'warm_start': True}
#Model:  MLP  Accuracy: 0.86  Best Parameters: {'activation': 'relu', 'alpha': 1e-08, 'early_stopping': 1, 'hidden_layer_sizes': 13, 'learning_rate': 'adaptive', 'max_iter': 200, 'random_state': 9, 'solver': 'lbfgs', 'verbose': 0}

# Aggregating the Best Results from different Algorithms

In [ ]:

sns.barplot(x='mod', y='weight', data=w).set(title= 'Model Weights')


In [ ]:
rf=w['weight'][w['mod']=='RF'].item()
xgb=w['weight'][w['mod']=='XGB'].item()
abc=w['weight'][w['mod']=='ABC'].item()
gbc=w['weight'][w['mod']=='GBC'].item()
mlp=w['weight'][w['mod']=='MLP'].item()
wmx=w['weight'].max()

In [ ]:
result

# Creating the Submission File

In [ ]:
result['Exited']= np.round_(((result.RF*rf)+(result.MLP*mlp)+(result.XGB*xgb)+(result.ABC*abc)+(result.GBC*gbc))/(5*wmx),decimals=1)

result

submission=result[['id','Exited']]
submission.to_csv('submission.csv', index=False)

In [ ]:
submission